## Imports

In [1]:
#standard useful libraries
import pandas as pd
import numpy as np
import random

#model selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

#neural network dependencies
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Loading Data

In [2]:
#Loading data from csv
pipes_df = pd.read_csv("Pipe_data.csv")
pipes_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (57,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,FID,MOID,DIAMETER,PIPE_LENGTH,MATERIAL,DESCRIPTION,YEAR_INSTALLED,PRESSURE_ZONE,LINING,LINE_YEAR,...,WORK_ORDER_NUMBER,Comments,ReportDate,StartDateT,CompletedD,ReportedY,SrvcRqst_1,MATERIAL_1,MUSYM,FAILURE
0,16420,P35881,8.0,861.090481,M.J.,Distribution,1952,Hemp 1175,,0,...,1143689,PER UNIT 819\n\rREQUEST 1 6X12 PLATE AND 4 BAG...,2014-10-25,2014-11-16,2014-11-16,2014,Water Request,CAS,Ub,1
1,44465,P88079,8.0,749.511184,TYT,Distribution,1965,Hemp 1175,,0,...,1311148,MAIN BREAK,2015-08-18,2015-08-18,2015-08-18,2015,Water Request,DIP,ReD,1
2,23810,P53764,8.0,171.554040,M.J.,Distribution,1958,Hemp 1175,,0,...,1323748,PER JONATHAN WEBB THERE IS A MAIN BREAK AT THI...,2015-09-11,2015-09-12,2015-09-12,2015,Water Pressure Complaint,DIP,Ub,1
3,52630,P400741,8.0,42.761097,M.J.,Distribution,1958,Hemp 1175,,0,...,1323748,PER JONATHAN WEBB THERE IS A MAIN BREAK AT THI...,2015-09-11,2015-09-12,2015-09-12,2015,Water Pressure Complaint,DIP,Ub,1
4,10510,P20210,12.0,31.300481,DIP,Transmission,2006,Hemp 1175,,0,...,1628288,PER J. SKINNER MAIN BREAK AT LOCATION\nCall ta...,2017-01-22,2017-02-28,2017-02-28,2017,Turn off burst pipe,DIP,Ub,1


In [3]:
#get dummies for columns where applicable--MATERIAL, DESCRIPTION, PRESSURE_ZONE
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['MATERIAL'], prefix='MATERIAL')], axis=1)
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['DESCRIPTION'], prefix='DESCRIPTION')], axis=1); 
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['PRESSURE_ZONE'], prefix='P-ZONE')], axis=1);

#drop original columns--MATERIAL, DESCRIPTION, PRESSURE_ZONE
pipes_df2 = pipes_df.drop(columns={ 'MATERIAL',
 'DESCRIPTION',
 'PRESSURE_ZONE',})

#Condense correlating PATTERN + DEMAND columns into 1 column
pipes_df2 = pipes_df2.rename(columns={"DEMAND1":"Demand_RES", "DEMAND2":"Demand_NON-RES","DEMAND3":"Demand_WHOLE","DEMAND4":"Demand_FLUSH","DEMAND5":"Demand_UFW",
                                     "DEMAND6":"Demand_RES-DISTR","DEMAND7":"Demand_NON RES-DISTR","DEMAND8":"Demand_2030-CWTP-NS"})

#Delete unneeded columns
#NOTE to Team: I didn’t see any data at all in the ‘DEMAND9’, ‘PATTERN9’, ‘DEMAND10’, ‘PATTERN10’ columns,so I deleted all of these columns
pipes_df3 = pipes_df2.drop(columns={"PATTERN1", "PATTERN2", "PATTERN3", "PATTERN4", "PATTERN5", "PATTERN6", "PATTERN7", "PATTERN8", "PATTERN9", "PATTERN10", "DEMAND9", "DEMAND10"})

In [4]:
#Create new Age column
#Age is being calculated as 2018 - installation year if no report was made
#Report year - installation year otherwise
tempAgeList = []
for row in np.arange(len(pipes_df3['FID'])):
    try:
        value = int(pipes_df3.iloc[row]['ReportDate'][:4]) - pipes_df3.iloc[row]['YEAR_INSTALLED']
    except:
        value = 2018 - pipes_df3.iloc[row]['YEAR_INSTALLED']
    tempAgeList.append(value)
pipes_df3['Age']=tempAgeList

In [5]:
#Add a column to be opposite of FAILURE column for use in neural network training
tempSuccessList = []
for row in np.arange(len(pipes_df3['FID'])):
    value = 1 - pipes_df3.iloc[row]['FAILURE']
    tempSuccessList.append(value)
pipes_df3['SUCCESS'] = tempSuccessList

# Machine Learning

# PreProcessing

In [6]:
#Select which columns to train off of.
#A number of iterations of this were initially used when exploring the data
#In the end it was decided to only use average values for those which have averages

inputColumns = ['DIAMETER', 'PIPE_LENGTH','ROUGHNESS', 'FLOW', 'VELOCITY', 'HEADLOSS',
       'HL1000', 'MAX_FLOW', 'MIN_FLOW', 'AVE_FLOW', 'MAX_VELOCITY',
       'MIN_VELOCITY', 'AVE_VELOCITY', 'MAX_HEADLOSS', 'MIN_HEADLOSS',
       'AVE_HEADLOSS', 'ELEVATION','RUN_ELEV', 'DEMAND',
       'HEAD', 'PRESSURE', 'MAX_HEAD', 'MIN_HEAD', 'AVE_HEAD', 'MAX_PRESS',
       'MIN_PRESS', 'AVE_PRESS', 'DIFF_PRESS','MATERIAL_AMTYT',
       'MATERIAL_CI', 'MATERIAL_CO', 'MATERIAL_COP', 'MATERIAL_CPP',
       'MATERIAL_Copper', 'MATERIAL_DEL', 'MATERIAL_DIP', 'MATERIAL_EARGEO',
       'MATERIAL_GP', 'MATERIAL_HDPE', 'MATERIAL_L.J.', 'MATERIAL_M.J.',
       'MATERIAL_MCW', 'MATERIAL_OTH', 'MATERIAL_PE', 'MATERIAL_PP',
       'MATERIAL_PVC', 'MATERIAL_R.L.J.', 'MATERIAL_RCP', 'MATERIAL_SP',
       'MATERIAL_ST', 'MATERIAL_T.D', 'MATERIAL_T.D.', 'MATERIAL_TTE',
       'MATERIAL_TYT', 'MATERIAL_UNK', 'MATERIAL_W.I.','Age']

In [7]:
#Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(pipes_df3[inputColumns],pipes_df3['FAILURE'],stratify=pipes_df3['FAILURE'],test_size = 0.3, random_state=42)

#Make another split for y which uses the additional 'SUCCESS' column for use in neural network
#Note: Using random state the X_train and X_test would be identical and so need not be kept
_, _, y_train_2, y_test_2 = train_test_split(pipes_df3[inputColumns],pipes_df3[['FAILURE','SUCCESS']],stratify=pipes_df3['FAILURE'],test_size = 0.3, random_state=42)

In [8]:
#Note: If we were to train and test on the split as it currently is, it will just guess that everything is an unbroken pipe
#This is not desired behavior

#Split X_train into a list of those where the pipes are broken and those which arent
#Then, undersample those pipes which are unbroken to avoid bias
X_train_false = X_train[y_train==0].sample(n=sum(y_train), random_state=42)
X_train_true = X_train[y_train==1].sample(n=sum(y_train), random_state=42)

#Recombine the sampled X_train into one list and keep only those y_train values corresponding to selected indices
X_train_sampled = X_train_true.append(X_train_false)
y_train_sampled = y_train.loc[X_train_sampled.index]
y_train_sampled_2 = y_train_2.loc[X_train_sampled.index]

#Apply min-max scalers to X
#Note: applying min-max too early changed format of dataframe into a np.array and interfered with the sampling process
X_minmax = MinMaxScaler().fit(X_train)
X_train_minmax = X_minmax.transform(X_train_sampled)
X_test_minmax = X_minmax.transform(X_test)

## Logistic Regression

In [9]:
#Set a logistic regression as a model
lrmodel = LogisticRegression()
lrmodel.fit(X_train_minmax,y_train_sampled)
lrmodel.score(X_train_minmax,y_train_sampled)

0.6892797319932998

In [10]:
lrmodel.score(X_test_minmax,y_test)

0.701142585098786

In [11]:
confusion_matrix(y_test,lrmodel.predict(X_test_minmax))

array([[11100,  4680],
       [  342,   682]], dtype=int64)

## Neural Network

In [12]:
#Add a shallow neural network
nnmodel = Sequential()

#Add layers
nnmodel.add(Dense(100, activation='relu', input_dim=X_train_minmax.shape[1]))
nnmodel.add(Dense(2,activation = 'softmax'))

#Compile
nnmodel.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

#Fit the training model to the data
nnmodel.fit(
    X_train_minmax,
    y_train_sampled_2,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 1s - loss: 0.6586 - acc: 0.6074
Epoch 2/100
 - 0s - loss: 0.6410 - acc: 0.6277
Epoch 3/100
 - 0s - loss: 0.6306 - acc: 0.6413
Epoch 4/100
 - 0s - loss: 0.6224 - acc: 0.6489
Epoch 5/100
 - 0s - loss: 0.6120 - acc: 0.6619
Epoch 6/100
 - 0s - loss: 0.6008 - acc: 0.6790
Epoch 7/100
 - 0s - loss: 0.5971 - acc: 0.6786
Epoch 8/100
 - 0s - loss: 0.5882 - acc: 0.6899
Epoch 9/100
 - 0s - loss: 0.5823 - acc: 0.6960
Epoch 10/100
 - 0s - loss: 0.5776 - acc: 0.7018
Epoch 11/100
 - 0s - loss: 0.5699 - acc: 0.7127
Epoch 12/100
 - 0s - loss: 0.5654 - acc: 0.7194
Epoch 13/100
 - 0s - loss: 0.5616 - acc: 0.7131
Epoch 14/100
 - 0s - loss: 0.5571 - acc: 0.7245
Epoch 15/100
 - 0s - loss: 0.5570 - acc: 0.7240
Epoch 16/100
 - 0s - loss: 0.5559 - acc: 0.7255
Epoch 17/100
 - 0s - loss: 0.5500 - acc: 0.7265
Epoch 18/100
 - 0s - loss: 0.5512 - acc: 0.7263
Epoch 19/100
 - 0s - loss: 0.5493 - acc: 0.7240
Epoch 20/100
 - 0s - loss: 0.5450 - acc: 0.7307
Epoch 21/100
 - 0s - loss: 0.5456 - acc: 0.7255
E

In [13]:
nnmodel_loss, nnmodel_accuracy = nnmodel.evaluate(X_test_minmax, y_test_2, verbose=2)
print(f"Loss: {nnmodel_loss}, Accuracy: {nnmodel_accuracy}")

Loss: 0.4486992735858192, Accuracy: 0.8023089740537968


## Random Forests

### Decide on number of estimators to use

In [14]:
#Load a Random Forest model
model = RandomForestClassifier()

#Set a parameter grid
#Only real parameters we needed to check was the number of estimators
param_grid = {'n_estimators': [20,50,100,200,300]}
grid = GridSearchCV(model, param_grid, verbose=3)

#Fit the grid
grid.fit(X_train_minmax,y_train_sampled)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] n_estimators=20 .................................................
[CV] ......... n_estimators=20, score=0.746859296482412, total=   0.2s
[CV] n_estimators=20 .................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] ........ n_estimators=20, score=0.7669597989949749, total=   0.3s
[CV] n_estimators=20 .................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.6s remaining:    0.0s


[CV] ......... n_estimators=20, score=0.760678391959799, total=   0.3s
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=50, score=0.7443467336683417, total=   0.7s
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=50, score=0.7682160804020101, total=   0.8s
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=50, score=0.7832914572864321, total=   0.8s
[CV] n_estimators=100 ................................................
[CV] ....... n_estimators=100, score=0.7675879396984925, total=   1.5s
[CV] n_estimators=100 ................................................
[CV] ....... n_estimators=100, score=0.7694723618090452, total=   1.5s
[CV] n_estimators=100 ................................................
[CV] ....... n_estimators=100, score=0.7845477386934674, total=   1.7s
[CV] n_estimators=200 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   36.3s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [20, 50, 100, 200, 300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [15]:
#Find the best score and best parameters
print(f'The best score was {grid.score(X_test_minmax,y_test)}\nThe best parameters were {grid.best_params_}')

The best score was 0.8056415139252558
The best parameters were {'n_estimators': 100}


In [16]:
#Take a look at classification report to confirm that the model is ideal
print(classification_report(y_test, grid.predict(X_test_minmax)))

             precision    recall  f1-score   support

          0       0.98      0.81      0.89     15780
          1       0.21      0.78      0.33      1024

avg / total       0.94      0.81      0.85     16804



In [17]:
#Take a look at the confusion matrix to confirm that there is no unintended behavior (e.g. only guessing unbroken)
confusion_matrix(y_test, grid.predict(X_test_minmax))

array([[12741,  3039],
       [  227,   797]], dtype=int64)

# Performing deeper analysis with Random Forests

In [18]:
#Apply cross validation so as to have percentage predictions for all data points without bias towards training data
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
split = cv.split(pipes_df3[inputColumns],pipes_df3['FAILURE'])
rfmodel = RandomForestClassifier(n_estimators=300, random_state=42)

In [19]:
#Create new column for use in displaying results of analysis to be filled in later
#Final results will be stored in final_df
scores = []
final_df = pipes_df3
final_df['BurstChance'] = 0
confusion = []

In [20]:
#For each train-test split from our crossvalidation, run the same type of analysis as before
for train_idx, test_idx in cv.split(pipes_df3[inputColumns],pipes_df3['FAILURE']):
    Current_X_train = pipes_df3[inputColumns].loc[train_idx]
    Current_X_test = pipes_df3[inputColumns].loc[test_idx]
    Current_y_train = pipes_df3['FAILURE'].loc[train_idx]
    Current_y_test = pipes_df3['FAILURE'].loc[test_idx]
    
    
    #Undersample training data as before
    Current_X_train_false = Current_X_train[Current_y_train==0].sample(n=sum(Current_y_train))
    Current_X_train_true = Current_X_train[Current_y_train==1].sample(n=sum(Current_y_train))
    
    Current_X_train_sampled = Current_X_train_true.append(Current_X_train_false)
    Current_y_train_sampled = Current_y_train.loc[Current_X_train_sampled.index]
    
    Current_X_minmax = MinMaxScaler().fit(Current_X_train)
    Current_X_train_minmax = Current_X_minmax.transform(Current_X_train_sampled)
    Current_X_test_minmax = Current_X_minmax.transform(Current_X_test)

    
    rfmodel.fit(Current_X_train_minmax, Current_y_train_sampled)
    score = rfmodel.score(Current_X_test_minmax, Current_y_test)
    scores.append(score)
    currentconfusion = confusion_matrix(Current_y_test,rfmodel.predict(Current_X_test_minmax))
    print(currentconfusion)
    confusion.append([currentconfusion[0][0],currentconfusion[0][1],currentconfusion[1][0],currentconfusion[1][1]])
    
    #Store prediction probabilities into final_df
    final_df['BurstChance'].loc[test_idx] = [x[1] for x in list(rfmodel.predict_proba(Current_X_test_minmax))]


[[8377 2143]
 [ 116  567]]


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[8495 2025]
 [ 135  548]]


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[8500 2020]
 [ 126  556]]


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[8390 2130]
 [ 140  542]]


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[8544 1976]
 [ 158  524]]


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [21]:
#Look at combined confusion matrix
truepositive = confusion[0][0] + confusion[1][0]+confusion[2][0]+confusion[3][0]+confusion[4][0]
falsenegative = confusion[0][1] + confusion[1][1]+confusion[2][1]+confusion[3][1]+confusion[4][1]
falsepositive = confusion[0][2] + confusion[1][2]+confusion[2][2]+confusion[3][2]+confusion[4][2]
truenegative = confusion[0][3] + confusion[1][3]+confusion[2][3]+confusion[3][3]+confusion[4][3]


In [22]:
print(f'[{[truepositive, falsenegative]}\n{[falsepositive,truenegative]}]')

[[42306, 10294]
[675, 2737]]


In [23]:
#Look at final scores for each
scores

[0.798357582790324,
 0.8071945014728198,
 0.8084270665952509,
 0.7973576147116587,
 0.8094983038743082]

In [24]:
#Output top ten most likely pipes to burst which are currently unbroken according to our model
final_df[['BurstChance','FID','MOID'] + inputColumns].loc[final_df['FAILURE']==0].sort_values('BurstChance', ascending=False).head(10)

,BurstChance,FID,MOID,DIAMETER,PIPE_LENGTH,ROUGHNESS,FLOW,VELOCITY,HEADLOSS,HL1000,...,MATERIAL_RCP,MATERIAL_SP,MATERIAL_ST,MATERIAL_T.D,MATERIAL_T.D.,MATERIAL_TTE,MATERIAL_TYT,MATERIAL_UNK,MATERIAL_W.I.,Age
919,0.973333,954,P103643,42.0,855.946881,130,32.3240,0.0075,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0,60
3783,0.973333,52675,P401104,8.0,498.518309,133,76.4881,0.4882,0.0747,0.1499,...,0,0,0,0,0,0,0,0,0,2
18637,0.970000,28712,P49568,6.0,485.640173,39,6.9836,0.0792,0.0341,0.0701,...,0,0,0,0,0,0,0,0,0,77
55769,0.970000,53927,P45877,2.0,328.264650,34,0.7676,0.0784,0.1049,0.3194,...,0,0,0,0,0,0,0,0,1,56
2146,0.966667,16996,P38945,6.0,651.419297,39,6.5829,0.0747,0.0410,0.0630,...,0,0,0,0,0,0,0,0,0,78
55878,0.963333,53929,P46994,2.0,311.496146,40,0.8835,0.0902,0.0956,0.3068,...,0,0,0,0,0,0,0,0,1,52
54023,0.960000,17710,P44896,6.0,624.176891,39,22.8996,0.2598,0.3948,0.6325,...,0,0,0,0,0,0,0,1,0,77
55987,0.960000,52331,P58814,2.0,292.452009,40,1.4956,0.1527,0.2378,0.8131,...,0,0,0,0,0,0,0,0,1,46
2746,0.956667,38641,P82895,6.0,565.153552,43,14.3589,0.1629,0.1256,0.2223,...,0,0,0,0,0,0,0,0,0,52
19048,0.956667,12177,P29247,6.0,507.967516,34,18.3472,0.2082,0.2748,0.5409,...,0,0,0,0,0,0,0,0,0,52


In [25]:
#Output most relevant parameters according to our recent model
print(sorted(zip(map(lambda x: round(x, 4), rfmodel.feature_importances_), inputColumns), reverse=True))

[(0.0899, 'PIPE_LENGTH'), (0.0845, 'Age'), (0.038, 'ROUGHNESS'), (0.0375, 'MAX_HEAD'), (0.0365, 'MAX_HEADLOSS'), (0.0355, 'DEMAND'), (0.0351, 'MIN_HEAD'), (0.0349, 'AVE_HEADLOSS'), (0.0346, 'AVE_HEAD'), (0.0332, 'HEAD'), (0.0315, 'DIFF_PRESS'), (0.0308, 'DIAMETER'), (0.029, 'FLOW'), (0.0282, 'MAX_FLOW'), (0.0282, 'HEADLOSS'), (0.0281, 'MIN_FLOW'), (0.0279, 'AVE_FLOW'), (0.0274, 'AVE_VELOCITY'), (0.0271, 'MIN_PRESS'), (0.0271, 'MAX_VELOCITY'), (0.0266, 'MIN_VELOCITY'), (0.0263, 'VELOCITY'), (0.0254, 'AVE_PRESS'), (0.0252, 'PRESSURE'), (0.025, 'HL1000'), (0.0249, 'ELEVATION'), (0.0247, 'RUN_ELEV'), (0.0246, 'MIN_HEADLOSS'), (0.0246, 'MAX_PRESS'), (0.0066, 'MATERIAL_M.J.'), (0.006, 'MATERIAL_T.D.'), (0.0047, 'MATERIAL_DIP'), (0.0033, 'MATERIAL_TYT'), (0.0018, 'MATERIAL_L.J.'), (0.0017, 'MATERIAL_DEL'), (0.0011, 'MATERIAL_UNK'), (0.0007, 'MATERIAL_CPP'), (0.0005, 'MATERIAL_W.I.'), (0.0005, 'MATERIAL_CO'), (0.0002, 'MATERIAL_ST'), (0.0002, 'MATERIAL_OTH'), (0.0002, 'MATERIAL_CI'), (0.0001, 

In [26]:
len(inputColumns)

57

In [27]:
print(classification_report(y_test,rfmodel.predict(X_test_minmax)))

             precision    recall  f1-score   support

          0       1.00      0.82      0.90     15780
          1       0.25      0.94      0.40      1024

avg / total       0.95      0.83      0.87     16804

